# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
print(mongo.list_database_names())

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
print(db.list_collection_names())

In [ ]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {}
new_restaurant["BusinessName"] = "Penang Flavours"
new_restaurant["BusinessType"] = "Restaurant/Cafe/Canteen"
new_restaurant["BusinessTypeID"] = ""
new_restaurant["AddressLine1"] = "Penang Flavours"
new_restaurant["AddressLine2"] = "146A Plumstead Rd"
new_restaurant["AddressLine3"] = "London"
new_restaurant["AddressLine4"] = ""
new_restaurant["PostCode"] = "SE18 7DY"
new_restaurant["Phone"] = ""
new_restaurant["LocalAuthorityCode"] = "511"
new_restaurant["LocalAuthorityName"] = "Greenwich"
new_restaurant["LocalAuthorityWebSite"] = "http://www.royalgreenwich.gov.uk"
new_restaurant["LocalAuthorityEmailAddress"] = "health@royalgreenwich.gov.uk"

# Nested dictionary for scores
new_restaurant["scores"] = {}
new_restaurant["scores"]["Hygiene"] = ""
new_restaurant["scores"]["Structural"] = ""
new_restaurant["scores"]["ConfidenceInManagement"] = ""

# More fields for the restaurant
new_restaurant["SchemeType"] = "FHRS"

# Nested dictionary for geocode
new_restaurant["geocode"] = {}
new_restaurant["geocode"]["longitude"] = "0.08384000"
new_restaurant["geocode"]["latitude"] = "51.49014200"

new_restaurant["RightToReply"] = ""
new_restaurant["Distance"] = 4623.9723280747176
new_restaurant["NewRatingPending"] = True

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Build the query step by step
query = {}
query['BusinessType'] = 'Restaurant/Cafe/Canteen'

# Define the fields to return, one by one
fields = {}
fields['BusinessTypeID'] = 1
fields['BusinessType'] = 1

# Execute the query and get the first matching document
result = establishments.find_one(query, projection=fields)

# Print the result using pprint
pprint(result)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
filter_query = {}
filter_query["BusinessName"] = "Penang Flavours"

# Build the update data step by step
update_data = {}
update_data["$set"] = {}
update_data["$set"]["BusinessTypeID"] = 1

# Execute the update operation
establishments.update_one(filter_query, update_data)


In [ ]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName":"Penang Flavours"}))

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {}
query['LocalAuthorityName'] = 'Dover'

# Initialize a counter variable
count = 0

# Iterate through the documents matching the query and count them manually
cursor = establishments.find(query)
for document in cursor:
    count += 1

# Store the final count in a variable and print it
Dover_documents = count
Dover_documents

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_query = {}
delete_query['LocalAuthorityName'] = 'Dover'

# Execute the delete operation using the query
result = establishments.delete_many(delete_query)

# Print the number of documents deleted (optional, but common for a novice to check)
deleted_count = result.deleted_count
print("Number of documents deleted:", deleted_count)

In [ ]:
# Check if any remaining documents include Dover
query = {}
query['LocalAuthorityName'] = 'Dover'

# Initialize a counter variable
count = 0

# Manually count the documents that match the query
cursor = establishments.find(query)
for doc in cursor:
    count += 1

# Store the count in a variable and print it
Dover_documents = count
Dover_documents

In [ ]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
update_query = {}
update_query['$set'] = {}
update_query['$set']["geocode.longitude"] = {}
update_query['$set']["geocode.longitude"]['$toDecimal'] = "$geocode.longitude"
update_query['$set']["geocode.latitude"] = {}
update_query['$set']["geocode.latitude"]['$toDecimal'] = "$geocode.latitude"

# Apply the update to all documents
result = establishments.update_many({}, [update_query])

# Print the number of documents modified (optional, for verification)
modified_count = result.modified_count
print("Number of documents modified:", modified_count

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
update_query = {}
update_query['$set'] = {}
update_query['$set']["RatingValue"] = {}
update_query['$set']["RatingValue"]['$toInt'] = "$RatingValue"

# Apply the update to all documents
result = establishments.update_many({}, [update_query])

# Print the number of documents modified (optional, for verification)
modified_count = result.modified_count
print("Number of documents modified:", modified_count)

In [ ]:
# Check that the coordinates and rating value are now numbers
pprint(establishments.find_one())